In [1]:
import pymysql  
import  pymysql.cursors 
#https://www.runoob.com/python/python-mysql.html
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import text 
engine=create_engine('mysql+pymysql://账户名:密码@IP地址:端口号/数据库名')

In [2]:
#https://www.runoob.com/python3/python3-mysql.html
import pymysql
db = pymysql.connect("IP","账户名","密码","数据库名" ) 
cursor = db.cursor()
sql_2="SHOW TABLES"
cursor.execute(sql_2)
result2=cursor.fetchall()
print('result2:',result2) 

db.close()

result2: (('dim_category',), ('dim_date',), ('dim_goods',), ('dim_member',), ('dim_shop',), ('fct_sales',), ('fct_sales_item',))


In [3]:
from sqlalchemy import create_engine
import pandas as pd
engine=create_engine('mysql+pymysql://账户名:密码@IP地址:端口号/数据库名')
sql = "SELECT * FROM fct_sales limit 8" 
# read_sql_query的两个参数: sql语句， 数据库连接
df_fct_sales = pd.read_sql_query(sql, engine)
(df_fct_sales.head(3))

,salesID,salesNo,dimDateID,dimShopID,dimMemberID,QTY,AMT,pAMT,mpAMT,updateTime
0,1,34131002542176,20170602,33,4821,1.000,7.99,0.0,0.210,2017-09-09 10:24:16
1,2,34131002542177,20170602,34,1730,1.000,227.00,0.0,12.000,2017-09-09 10:24:16
2,3,34131002542178,20170602,33,513,5.046,75.64,0.0,3.022,2017-09-09 10:24:16


/*
 * 多表操作之：表的行合并
 * 用union 
 * 1、合并的表的列数必须是一致，也就是合并的列数相同
 * 2、union 与union all 对二个查询结果合并的时候，如果有结果有重复，union 会把重复的值 删除 ；
 * 3、union 与union all 不对与order by  同时使用，所以如果你要对结果进行排序，最好用子查询
 */
-- 统计20170703-20170710周内每天及本周累计销售金额、订单量、会员数、订单占比

In [4]:
#第一步  
sql =text(""" select date_format(dimDateID,'%W') as week_day
      ,sum(AMT) as sales_money
      ,count(distinct salesID) as order_number
      ,count(distinct dimMemberID) as member_number
      ,count(distinct salesID)/(select count(distinct salesID) from dw.fct_sales
								where dimDateID between '20170703' and '20170710') as order_rate
from dw.fct_sales
where dimDateID between '20170703' and '20170710'
group by date_format(dimDateID,'%W');""")
df= pd.read_sql_query(sql, engine)
df

,week_day,sales_money,order_number,member_number,order_rate
0,Friday,96602.58,1739,827,0.1172
1,Monday,236153.77,4231,1637,0.2851
2,Saturday,145862.13,1988,871,0.1340
3,Sunday,128228.18,2016,926,0.1358
4,Thursday,116434.51,1805,802,0.1216
5,Tuesday,63387.56,1201,612,0.0809
6,Wednesday,101509.87,1860,886,0.1253


In [5]:
sql =text("""
select dimMemberID
       ,sum(AMT) as money
       ,(select sum(amt) from dw.fct_sales where dimDateID='20170702' and dimMemberID<>0) as total_money
       ,sum(AMT)/ (select sum(amt) from dw.fct_sales where dimDateID='20170702' and dimMemberID<>0) as member_rate
from dw.fct_sales 
where dimDateID = '20170702'
and dimMemberID<>0
group by dimMemberID
order by sum(AMT) desc;
""")
# read_sql_query的两个参数: sql语句， 数据库连接
df = pd.read_sql_query(sql, engine)
# 输出employee表的查询结果
print(df.head(3))

   dimMemberID    money  total_money  member_rate
0         3110  3300.50     78204.66     0.042203
1         8207  2140.40     78204.66     0.027369
2         1677  1912.31     78204.66     0.024453


In [6]:
#第二部， 根据会员，查询消费记录
sql =text("""-- 比例希望是百分比
select  dimMemberID
       ,sum(AMT) as money
       ,(select sum(amt) from dw.fct_sales where dimDateID='20170702' and dimMemberID<>0) as total_money
       ,sum(AMT)/ (select sum(amt) from dw.fct_sales where dimDateID='20170702' and dimMemberID<>0) as member_rate
       ,concat(sum(AMT)/ (select sum(amt) from dw.fct_sales where dimDateID='20170702' and dimMemberID<>0)*100,'%') as member_rate_100
from dw.fct_sales 
where dimDateID = '20170702'
and dimMemberID<>0
group by dimMemberID
order by sum(AMT) desc;
""")
df = pd.read_sql_query(sql, engine)
print(df.head(3))

   dimMemberID    money  total_money  member_rate member_rate_100
0         3110  3300.50     78204.66     0.042203    4.220336742%
1         8207  2140.40     78204.66     0.027369    2.736921304%
2         1677  1912.31     78204.66     0.024453    2.445263492%


In [7]:
from sqlalchemy import create_engine
import pandas as pd
engine=create_engine('mysql+pymysql://账户名:密码@IP地址:端口号/数据库名')
sql ="""select dimMemberID
      ,money
      ,case when money<100 then 'D'
            when money>=100 and money<500 then 'C'
            when money>=500 and money<1000 then 'B'
            when money>=1000  then 'A'
       end as type_money
from 
(select dimMemberID
       ,sum(AMT) as money
from dw.fct_sales 
where dimDateID = '20170702'
and dimMemberID<>0
group by dimMemberID
order by sum(AMT) desc ) as t; 
"""
df= pd.read_sql_query(sql, engine)
# 输出employee表的查询结果
print(df.head(3))

   dimMemberID    money type_money
0         3110  3300.50          A
1         8207  2140.40          A
2         1677  1912.31          A


In [8]:
#作业 对7月份某周每位会员累计购买金额进行分段统计，并统计每个段的会员数
#第一步 对数据探索,了解有哪些列和含义
from sqlalchemy import create_engine
import pandas as pd
engine=create_engine('mysql+pymysql://账户名:密码@IP地址:端口号/数据库名')
sql = """SELECT * FROM fct_sales  """
df = pd.read_sql_query(sql, engine)
df.head(3) #.head(300)

,salesID,salesNo,dimDateID,dimShopID,dimMemberID,QTY,AMT,pAMT,mpAMT,updateTime
0,1,34131002542176,20170602,33,4821,1.000,7.99,0.0,0.210,2017-09-09 10:24:16
1,2,34131002542177,20170602,34,1730,1.000,227.00,0.0,12.000,2017-09-09 10:24:16
2,3,34131002542178,20170602,33,513,5.046,75.64,0.0,3.022,2017-09-09 10:24:16


In [9]:
# 第二步 先统计每个会员的累计购买金额,衍生出了 money 变量
sql =text("""select dimMemberID
       ,sum(AMT) as money
from dw.fct_sales 
where dimDateID between '20170702' and '20170709'
and dimMemberID<>0
group by dimMemberID
order by sum(AMT) desc;""")
df = pd.read_sql_query(sql, engine)
print(df.head(6))  #.head(300)
print(df.shape)

   dimMemberID     money
0         4976  11939.06
1         3110  11434.38
2         5366  11141.39
3         1677   9019.30
4          516   6487.61
5         5488   6080.59
(4169, 2)


In [10]:
# 第三步 将上一步的结果再提取，进行分频 ，，这里直接调用了衍生变量money  ,使用 case when end as
sql =text("""select dimMemberID
      ,money                       
      ,case when money<100 then 'D'
            when money>=100 and money<500 then 'C'
            when money>=500 and money<1000 then 'B'
            when money>=1000  then 'A'
       end as type_money
       from 
       (select dimMemberID,sum(AMT) as money
        from dw.fct_sales 
        where dimDateID between '20170702' and '20170709'
        and dimMemberID<>0
        group by dimMemberID
        order by sum(AMT) desc) 
        as temp1
""")
df = pd.read_sql_query(sql, engine)
df.head(3) #.head(300)

,dimMemberID,money,type_money
0,4976,11939.06,A
1,3110,11434.38,A
2,5366,11141.39,A


In [11]:
sql =text("""select dimMemberID,count(type_money),money,type_money
        from
        ( select dimMemberID,money,
            case when money<100 then 'D'
            when money>=100 and money<500 then 'C'
            when money>=500 and money<1000 then 'B'
            when money>=1000  then 'A'
               end as type_money
           from 
            (select dimMemberID,sum(AMT) as money
             from dw.fct_sales 
            where dimDateID between '20170702' and '20170709'
            and dimMemberID<>0
            group by dimMemberID
            order by sum(AMT) desc) 
            as temp1
             ) 
            as temp2
           group by dimMemberID,type_money
           """)
df = pd.read_sql_query(sql, engine) 
df.head(60) 

,dimMemberID,count(type_money),money,type_money
0,4976,1,11939.06,A
1,3110,1,11434.38,A
2,5366,1,11141.39,A
3,1677,1,9019.30,A
4,516,1,6487.61,A
5,5488,1,6080.59,A
6,5477,1,5550.83,A
7,7254,1,4548.96,A
8,3619,1,4506.96,A
9,2103,1,4480.00,A


In [12]:
sql =text("""select count(type_money),type_money
        from
        ( select dimMemberID,money,
            case when money<100 then 'D'
            when money>=100 and money<500 then 'C'
            when money>=500 and money<1000 then 'B'
            when money>=1000  then 'A'
               end as type_money
           from 
            (select dimMemberID,sum(AMT) as money
             from dw.fct_sales 
            where dimDateID between '20170702' and '20170709'
            and dimMemberID<>0
            group by dimMemberID
            order by sum(AMT) desc) 
            as temp1
             ) 
            as temp2
           group by type_money
           """)
df_fct_sales_total = pd.read_sql_query(sql, engine)
df_fct_sales_total.head(60) #.head(300)

,count(type_money),type_money
0,54,A
1,82,B
2,1385,C
3,2648,D


In [16]:
from sqlalchemy import create_engine
import pandas as pd
engine=create_engine('mysql+pymysql://账户名:密码@IP地址:端口号/数据库名')
sql =text( """SELECT COUNT(AMT) FROM fct_sales  
        WHERE week(date_format(dimDateID,'%Y-%m-%d'))=28""")
df_fct_sales_total = pd.read_sql_query(sql, engine)
print(df_fct_sales_total)  #.head(300)

   COUNT(AMT)
0       13299
